In [16]:
.libPaths()


[1] "/home/jpm73279/.conda/envs/r_env/lib/R/library"

In [18]:
library("here")
library(devtools)
library(Seurat)
library(harmony)
library(RcppML)
library(reshape2)
library(tidyverse)
load_all('/home/jpm73279/Socrates')
data_path <- "/scratch/jpm73279/comparative_single_cell/02.QC_clustering/zea_mays/QC_output/"

set.seed(12345)



ℹ Loading Socrates

Warning message in (function (dep_name, dep_ver = "*") :
"Dependency package 'cicero' not available."


ERROR: [1m[33mError[39m in [1m[1m`abort_for_missing_packages()`:[22m
[33m![39m Dependency package(s) 'cicero' not available.


In [8]:

library(randomcoloR)
other <- randomcoloR::distinctColorPalette(k = 55)
other <- sample(other)

In [9]:
# bed <- system.file("extdata", "test.tn5.bed.gz", package = "Socrates")
#ann <- system.file("extdata", "gencode.v19.annotation.gff3.gz", package = "Socrates")
#chr <- system.file("extdata", "hg19.txt", package = "Socrates")

reference_genome_path <- "/home/jpm73279/genome_downloads/Zm-B73-REFERENCE-NAM-5.0/"
#analysis_path <- "/scratch/jpm73279/comparative_single_cell/01.alignments_annotations/zea_mays/"
ann <- paste0(reference_genome_path,"Zm-B73-REFERENCE-NAM-5.0_Zm00001eb.1.gff3")
chr <- paste0(reference_genome_path,"Zm-B73-REFERENCE-NAM-5.0.chrom.size")
bed_intersections <- "/scratch/jpm73279/comparative_single_cell/01.alignments_annotations/zea_mays/Zm.all_mergeed_repds.unique.mpq10.tn5.sorted.bed"


#Load Object
Zm.subcluster <- loadBEDandGenomeData(bed_intersections, ann, chr)
#zea_m.rep.peaks  <- paste0(pre, "macs2_temp", sep=".")

ERROR: Error in loadBEDandGenomeData(bed_intersections, ann, chr): could not find function "loadBEDandGenomeData"


In [ ]:

marker_list_maize_file <- "/home/jpm73279/genome_downloads/Zm-B73-REFERENCE-NAM-5.0/markers/Zm.markers.leaf.txt"
marker_list_maize <- read_delim(marker_list_maize_file, delim='\t', 
                                       col_names = TRUE)

In [ ]:
loaded_socrates_data <- readRDS("/scratch/jpm73279/comparative_single_cell/02.QC_clustering/zea_mays/clustering/Zm_leaf.merged_replicates.updated_annotation_approach.V3.2022-06-23.saved_socrates_output.RDS")

In [ ]:
loaded_socrates_data$Clusters

In [ ]:
loaded_socrates_data$Clusters  %>% 
        group_by(Louvain_cluster_safe)  %>% 
    summarise(n())
    

## Clustering on Annotation Peaks from V3

In [ ]:
library(cowplot)
library(randomcoloR)
options(repr.plot.width=15, repr.plot.height=15)
other <- randomcoloR::distinctColorPalette(k = 55)
other <- sample(other)



In [ ]:
Zm.subcluster$acr <- read.table("/scratch/jpm73279/comparative_single_cell/02.QC_clustering/zea_mays/clustering/LouvainClustering_bws/Zm.LouvainCluster.V3.500bp_peaks.fixed.bed")

In [ ]:
?generateMatrix

In [ ]:
Zm.subcluster <- generateMatrix(Zm.subcluster, 
                                              filtered=FALSE,
                                              peaks = TRUE,
                                            
                                              verbose=TRUE)

In [ ]:
updated_meta <- loaded_socrates_data$Clusters
rownames(updated_meta) <- updated_meta$cellID
Zm.subcluster$meta <- updated_meta


In [ ]:
Zm.subcluster.sparse <- convertSparseData(Zm.subcluster, verbose = TRUE)

In [ ]:
# clean sparse counts matrix ---------------------------------------------
Zm.subcluster.matrix_counts.clean <- cleanData(Zm.subcluster.sparse, 
                     min.t=0.005,  # minimum feature frequency across cells
                     max.t=0.1, # maximum feature frequency across cells
                     verbose=T)


In [ ]:
# normalize with TFIDF ---------------------------------------------------
Zm.subcluster.tfidf <- tfidf(Zm.subcluster.matrix_counts.clean)

In [ ]:
# project with NMF -------------------------------------------------------
Zm.subcluster.reduce_dim <- reduceDims(Zm.subcluster.tfidf,
                      method="SVD", 
                      n.pcs=30, 
                      num.var=50000,
                      svd_slotName="SVD")

In [ ]:
# reduce to 2-dimensions with UMAP ---------------------------------------
Zm.subcluster.tfidf <- projectUMAP(Zm.subcluster.reduce_dim, 
                       verbose=T,
                       k.near=15,
                       m.dist=0.01,
                       svd_slotName="SVD")

library(randomcoloR)
other <- randomcoloR::distinctColorPalette(k = 55)
other <- sample(other)

ggplot(Zm.subcluster.tfidf$meta, aes(x=umap1, y = umap2, color = LouvainClusters)) +
  scale_colour_manual(values=other)  + guides(colour = guide_legend(override.aes = list(size=5))) + 
 geom_point(size = .25, alpha = .8)

In [ ]:
# identify clusters using neighborhood graph -----------------------------
tfidf.obj.dob.filt.clusters <- callClusters(Zm.subcluster.tfidf, 
                        res=1.6,
                        k.near = 30,
                        #min.reads=1000,
                        verbose=T,
                        svd_slotName= "SVD",
                        cluster_slotName="sub_cluster",
                        cleanCluster=FALSE)


In [ ]:
colnames(Zm.subcluster$meta)

In [ ]:
zea_annotation <- read_tsv("/home/jpm73279/Mendieta_et_al_comparative_single_cell/metrics/annotations/Zm_leaf.V3_final.txt")

In [ ]:
subset_zea <- zea_annotation  %>% 
    dplyr::select(cellID, V3_annotation_n)


combined_new_peaks_clustering_annotation <- left_join(tfidf.obj.dob.filt.clusters$sub_cluster, subset_zea, by = c("cellID"))

In [ ]:
options(repr.plot.width=20, repr.plot.height=5)

old_clusters <- ggplot(zea_annotation, aes(x=umap1, y = umap2, color = V3_annotation_n)) +
  scale_colour_manual(values=other)  + guides(colour = guide_legend(override.aes = list(size=5))) + 
 geom_point(size = .25, alpha = .8)+ ggtitle("Original Clustering w/Annotation")

new_clusters <- ggplot(combined_new_peaks_clustering_annotation, aes(x=umap1, y = umap2, color = V3_annotation_n)) +
  scale_colour_manual(values=other)  + guides(colour = guide_legend(override.aes = list(size=5))) + 
 geom_point(size = .25, alpha = .8) + ggtitle("Peak Clustering w/Annotation")


old_clusters + new_clusters 

In [25]:
nrow(zea_annotation)
nrow(combined_new_peaks_clustering_annotation)

[1] 15793

[1] 15202

In [26]:
prefix <- "Zm_leaf.merged_replicates.acr_bins"
output_location <- "/scratch/jpm73279/comparative_single_cell/02.QC_clustering/zea_mays/clustering/"

nmf.rd <- tfidf.obj.dob.filt.clusters$SVD
write.table(nmf.rd, file=paste0(output_location, prefix, ".SVD.full.reduced_dimensions.txt"), quote=F, row.names=T, col.names=T, sep="\t")


In [ ]:
call_per_cluster_ACRs <- function(meta_obj, 
                                  slot_name, 
                                  cluster_name, 
                                  bed_integrations, 
                                  base_output_name="test_Rrun", 
                                  output_dir="test", 
                                  genome_file,
                                  verbose=FALSE){
    
    clust_name <- c(cluster_name)
    
    cell_converted_trible <- as_tibble(meta_obj[[slot_name]])
    
    
    generate_count_data <- cell_converted_trible  %>% 
        group_by(!!sym(clust_name))  %>% 
        summarise(counts_per_clust = n())
    
    print(generate_count_data)
    
    final_tribble <- left_join(cell_converted_trible, generate_count_data)  %>% 
        mutate(cluster_name_final = str_c(!!sym(clust_name), "_ncell_", counts_per_clust))

    
    write_delim(final_tribble, "test_output_name.2.txt", 
            col_names = TRUE, quote = "none", delim = "\t")

    if(verbose){message("Calling peaks for Clusters - This may take some time. Check output terminal for info.")}
    generate_run_command = paste0("python /home/jpm73279/Mendieta_et_al_comparative_single_cell/python_scripts/scATAC/call_scACRs.py -bed ", bed_integrations, 
                                  " -meta ", "test_output_name.2.txt", 
                                  " -col ", "cluster_name_final", 
                                  " -base ", base_output_name, 
                                  " -outdir ", output_dir, 
                                  " -bw yes -fai ", genome_file)
    
    print(generate_run_command)
    system(generate_run_command)

    
    if(verbose){message("Done!")}
    
    output_peak_file_name <- paste0(output_dir, "/", base_output_name, ".500bp_peaks.bed")
    
    meta_obj$acr_file_name <- output_peak_file_name
    return(meta_obj)
    
}
